In [10]:
# ============================================================
# 03_fine_tuning.ipynb
# Fine-tuning du modèle InceptionV3 sur les fleurs
# ============================================================

# ------------------------------------------------------------
# 1. Importations
# ------------------------------------------------------------
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle


In [11]:
# ------------------------------------------------------------
# 2. Chemins et dossiers
# ------------------------------------------------------------
DATASET_PATH = "../data/flower_images"
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_BASELINE_PATH = os.path.join(MODEL_DIR, "inception_baseline.h5")

# Nom des fichiers avec timestamp pour versionner
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_FINETUNED_PATH = os.path.join(MODEL_DIR, f"inception_finetuned_{timestamp}.h5")
HISTORY_FINETUNE_PATH = os.path.join(MODEL_DIR, f"history_finetune_{timestamp}.pkl")

print("Dataset :", DATASET_PATH)
print("Classes :", os.listdir(DATASET_PATH))


Dataset : ../data/flower_images
Classes : ['Lilly', 'Lotus', 'Orchid', 'Sunflower', 'Tulip']


In [12]:
# ------------------------------------------------------------
# 3. Paramètres
# ------------------------------------------------------------
IMG_SIZE = (299, 299)
BATCH_SIZE = 32
EPOCHS = 25
LR = 1e-5  # très faible pour fine-tuning


In [13]:
# ------------------------------------------------------------
# 4. Générateurs de données
# ------------------------------------------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [ ]:
# ------------------------------------------------------------
# 5. Charger le modèle baseline
# ------------------------------------------------------------
print("\nChargement du modèle baseline...")
baseline_model = load_model(MODEL_BASELINE_PATH)
baseline_model.summary()


In [ ]:
# ------------------------------------------------------------
# 6. Déverrouiller certaines couches (fine-tuning)
# ------------------------------------------------------------
print("\nDégel des couches profondes pour fine-tuning...\n")
fine_tune_at = 250  # laisser 250 premières couches gelées

for layer in baseline_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in baseline_model.layers[fine_tune_at:]:
    layer.trainable = True


In [20]:
# ------------------------------------------------------------
# 7. Compilation
# ------------------------------------------------------------
baseline_model.compile(
    optimizer=Adam(learning_rate=LR),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [21]:
# ------------------------------------------------------------
# 8. Callbacks pour entraîner efficacement
# ------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)


In [ ]:
# ------------------------------------------------------------
# 9. Entraînement du modèle fine-tuné
# ------------------------------------------------------------
history_finetune = baseline_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)


In [ ]:
# ------------------------------------------------------------
# 10. Sauvegarde modèle et historique
# ------------------------------------------------------------
baseline_model.save(MODEL_FINETUNED_PATH)
with open(HISTORY_FINETUNE_PATH, "wb") as f:
    pickle.dump(history_finetune.history, f)

print("\n✔ Modèle fine-tuné sauvegardé :", MODEL_FINETUNED_PATH)
print("✔ Historique sauvegardé :", HISTORY_FINETUNE_PATH)


In [ ]:
# ------------------------------------------------------------
# 11. Visualisation
# ------------------------------------------------------------
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history_finetune.history["accuracy"], label="Train Acc FT")
plt.plot(history_finetune.history["val_accuracy"], label="Val Acc FT")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Fine-Tuning Accuracy")
plt.legend()
plt.grid()

# Loss
plt.subplot(1,2,2)
plt.plot(history_finetune.history["loss"], label="Train Loss FT")
plt.plot(history_finetune.history["val_loss"], label="Val Loss FT")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Fine-Tuning Loss")
plt.legend()
plt.grid()

plt.show()
